In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

#my helper functions from ipynb file
from ipynb.fs.full.PostgreHelper import * 

# Extract

In [16]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'

#extract all tables from the sheet
student_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Students"
df_student = pd.read_csv(student_url)

st_course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Student_Courses"
df_st_course = pd.read_csv(st_course_url)

course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Courses_Info"
df_course_info = pd.read_csv(course_url)

payment_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Payment_Method"
df_payment = pd.read_csv(payment_url)

purchased_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Purchased_Info"
df_purchase = pd.read_csv(purchased_url)

In [17]:
df_purchase

,student_id,course_id,payment_method_id,purchased_date
0,C001,COURSE001,P001,10/4/2021
1,C002,COURSE002,P002,10/5/2021
2,C003,COURSE003,P003,10/6/2021
3,C004,COURSE004,P004,10/7/2021
4,C005,COURSE005,P005,10/8/2021
5,C006,COURSE006,P006,10/9/2021
6,C007,COURSE007,P007,10/10/2021
7,C008,COURSE008,P008,10/11/2021
8,C009,COURSE009,P009,10/12/2021
9,C010,COURSE010,P010,10/13/2021


# Transform (Dimensional Model)

# Load

In [ ]:
#connect to my Postgre database that hosted on ElephantSql with free instance
connection, cursor = connect(host = 'arjuna.db.elephantsql.com', 
                             database = '*******', 
                             user = '*******', 
                             password = '*******', 
                             port = '5432')

In [ ]:
#create new table for the above youtube data
query = 'CREATE TABLE youtube_channel \
        (id SERIAL PRIMARY KEY, title text, url text, liked int, disliked int, views int, comment_count int); '
create_table(connection, cursor, query)

In [ ]:
#inserting the dataframe into youtube_channel table

#convert dataframe in tuples to iterate
tuples = [tuple(x) for x in df.to_numpy()]

#df columns into string
cols = ','.join(list(df.columns))

#insert query
query = "INSERT INTO youtube_channel(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

#inserting with executemany function
try:
    cursor.executemany(query, tuples)
    print("Data inserted using execute_many() successfully!")
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    cursor.close()

In [ ]:
#read PostgreSQL table into pandas dataframe
data = pd.read_sql_query(con = connection, sql = 'SELECT * FROM youtube_channel')
data.head(10)